---
## 1. String Operators Overview

KQL provides rich string matching capabilities optimized for different search patterns. Choose the operator based on your use case:

| Category | Operators | Use Case | Performance |
|----------|-----------|----------|-------------|
| **Word-Based** | `has`, `!has`, `has_all`, `has_any` | Tokenized word matching | Fast (indexed) |
| **Substring** | `contains`, `!contains`, `in()` | Substring search | Medium |
| **Case-Sensitive** | `==`, `!=`, `<`, `>` | Exact matching | Very Fast |
| **Case-Insensitive** | `=~`, `!~` | Pattern matching | Medium |
| **Regex** | `matches regex` | Complex patterns | Slow (use sparingly) |
| **Prefix/Suffix** | `startswith`, `endswith`, `has_prefix` | Path/URI matching | Fast |

**Pro Tip**: Use indexed operators (`has`, `==`, `has_prefix`) when possible for 10-100x performance improvement over substring search.

---
## 2. Word/Token-Based Operators

These operators treat fields as space-delimited word lists. Ideal for free-text description fields.

### `has` - Check if field contains word
- **Syntax**: `field has "word"`
- **Behavior**: Case-insensitive, word boundaries required (space/punctuation)
- **Performance**: Indexed, very fast

### `has_all` - Check if field contains ALL words
- **Syntax**: `field has_all ("word1", "word2")`
- **Behavior**: Unordered, case-insensitive
- **Use Case**: Multi-word problem descriptions

### `has_any` - Check if field contains ANY word
- **Syntax**: `field has_any ("word1", "word2")`
- **Behavior**: Unordered, case-insensitive
- **Use Case**: Multiple keywords for similar issues

### Real-World Example

```kql
// Find support cases mentioning both "service" and "timeout"
GetSCIMIncidentV2
| where CreatedDateTime > ago(7d)
| where Title has_all ("service", "timeout")
| project ServiceRequestNumber, Title, Severity, CreatedDateTime
| take 10

// Find cases with any emergency keywords
GetSCIMIncidentV2
| where Severity in ("1", "A")  // High severity
| where Description has_any ("critical", "down", "outage", "urgent")
| summarize CaseCount = dcount(ServiceRequestNumber) by Status
```

**Performance Notes**:
- `has` is indexed: ~10-100ms for 1M rows
- `has_all`/`has_any` are slower: use filters before applying
- Chain multiple filters: `| where field1 has "word1" | where field2 has "word2"` faster than `has_all`

---
## 3. Substring & Partial Match Operators

### `contains` - Substring search (any position)
- **Syntax**: `field contains "substring"`
- **Behavior**: Case-insensitive substring anywhere in field
- **Performance**: Not indexed, slower (~100-1000ms for 1M rows)
- **Use Case**: Searching partial product names, error codes

### `!contains` - Negation
- **Syntax**: `field !contains "substring"`
- **Behavior**: Field does NOT contain substring
- **Caution**: More expensive than `contains` (must scan all rows)

### `in()` - List membership
- **Syntax**: `field in ("val1", "val2", "val3")`
- **Behavior**: Exact match against list
- **Performance**: Very fast with small lists (<100 items)
- **Best Practice**: Prefer over multiple `contains` clauses

### Real-World Examples

```kql
// Find cases with "SQL" anywhere in product name
GetSCIMIncidentV2
| where DerivedProductName contains "SQL"
| summarize CaseCount = dcount(ServiceRequestNumber) by DerivedProductName

// Find cases NOT related to networking
GetSCIMIncidentV2
| where !Description contains "network" and !Description contains "firewall"
| summarize CaseCount = dcount(ServiceRequestNumber) by Severity

// Find cases for specific products (prefer over multiple contains)
GetSCIMIncidentV2
| where DerivedProductName in ("Exchange", "Teams", "OneDrive", "SharePoint")
| summarize AvgDaysToClose = avg(DaysToClose) by DerivedProductName
```

**Performance Best Practices**:
- Use `in()` for 3+ values instead of chained `contains`
- Apply `contains` after other filters to reduce row count
- Example: `| where CreatedDateTime > ago(7d) | where Description contains "error"` faster than reversed order

---
## 4. Case Sensitivity & Comparison Operators

### `==` - Exact match (case-sensitive)
- **Syntax**: `field == "value"`
- **Behavior**: Character-for-character match
- **Performance**: Indexed, very fast (~1-10ms)
- **Use Case**: Status fields, enums, IDs

### `!=` - Not equal (case-sensitive)
- **Syntax**: `field != "value"`
- **Performance**: Indexed, very fast

### `=~` - Case-insensitive match
- **Syntax**: `field =~ "pattern"`
- **Behavior**: Case-insensitive exact match
- **Performance**: Not indexed, medium (~50-500ms)

### `!~` - Case-insensitive negation
- **Syntax**: `field !~ "pattern"`
- **Performance**: Not indexed, medium

### Comparison Operators
- `<`, `<=`, `>`, `>=` for numeric and datetime values
- String comparison: lexicographic order (A < B < Z < a < b < z)

### Real-World Examples

```kql
// Case-sensitive status check (fast)
GetSCIMIncidentV2
| where Status == "Active"  // Exact match only
| summarize count()

// Case-insensitive product match (slower)
GetSCIMIncidentV2
| where DerivedProductName =~ "sharepoint"
| summarize count()

// Exclude specific statuses
GetSCIMIncidentV2
| where Status != "Closed" and Status != "Cancelled"
| summarize CaseCount = dcount(ServiceRequestNumber) by AgentAlias

// Datetime comparison
GetSCIMIncidentV2
| where CreatedDateTime >= datetime(2024-12-01) and CreatedDateTime < datetime(2025-01-01)
| summarize AvgResolutionTime = avg(DaysToClose) by Severity
```

**Best Practices**:
- Use `==` for Status/enum fields (indexed)
- Use `=~` sparingly; convert data types upfront if possible
- Example: `| extend NormalizedStatus = tolower(Status) | where NormalizedStatus == "active"` for repeated comparisons

---
## 5. Regex & Pattern Matching

### `matches regex` - Full regex support
- **Syntax**: `field matches regex "pattern"`
- **Behavior**: Perl-compatible regex (PCRE)
- **Performance**: SLOW (~500-5000ms for 1M rows) - use as last resort
- **Use Case**: Complex patterns (emails, IP addresses, error codes)

### Common Regex Patterns

| Pattern | Meaning | Example |
|---------|---------|----------|
| `^` | Start of string | `^ERROR` matches "ERROR: " |
| `$` | End of string | `\.log$` matches "app.log" |
| `.` | Any character | `E.ROR` matches "ERROR" |
| `*` | 0+ repetitions | `ERR.*` matches "ERROR", "ERR" |
| `+` | 1+ repetitions | `[0-9]+` matches "123" |
| `\d` | Digit [0-9] | `\d{3}-\d{3}-\d{4}` |
| `\w` | Word char [a-zA-Z0-9_] | `\w+@\w+` |
| `\|` | OR | `ERROR\|WARN\|CRITICAL` |

### Real-World Examples

```kql
// Find error codes in format ERR-1234
GetSCIMIncidentV2
| where Description matches regex @"ERR-\d{4}"
| extract "ERR-(?<ErrorCode>\d{4})" of Description
| summarize CaseCount = dcount(ServiceRequestNumber) by ErrorCode

// Find cases with critical severity keywords
GetSCIMIncidentV2
| where Title matches regex @"(CRITICAL|DOWN|OUTAGE|EMERGENCY)"
| summarize count()

// Parse error ID pattern: [PROD-123-ABC]
GetSCIMIncidentV2
| where Description matches regex @"\[\w+-\d+-\w+\]"
| extend ErrorPattern = extract(@"\[(.*?)\]", 1, Description)
| summarize count() by ErrorPattern

// Filter by complex phone number format
GetSCIMIncidentV2
| where CustomerPhone matches regex @"^\+?1?\d{10}$"
| summarize count()
```

**Performance Warning**:
- Apply `matches regex` AFTER all other filters
- For email validation: `contains "@"` is 100x faster than regex
- Consider pre-processing data with `extract()` to create indexed columns

---
## 6. Equality & List Operators

### `startswith` & `endswith` - Boundary matching
- **Syntax**: `field startswith "prefix"` or `field endswith ".txt"`
- **Performance**: Indexed, fast (~10-100ms)
- **Use Case**: File extensions, URI prefixes, SAP hierarchy paths

### `has_prefix` - Prefix matching (more flexible)
- **Syntax**: `field has_prefix "prefix"`
- **Behavior**: Word-boundary aware, case-insensitive
- **Use Case**: SAP path hierarchies (CDOM > Account > Product)

### Comparison Summary

| Operator | Case-Sens | Indexed | Boundary | Use Case |
|----------|-----------|---------|----------|----------|
| `==` | Yes | Yes | Exact | Status, IDs |
| `=~` | No | No | Exact | Case-insensitive enum |
| `contains` | No | No | Substring | Partial match |
| `has` | No | Yes | Word | Free-text search |
| `startswith` | Yes | Yes | Prefix | File ext, paths |
| `endswith` | Yes | Yes | Suffix | File ext |
| `has_prefix` | No | Yes | Word prefix | SAP paths |
| `matches regex` | Varies | No | Pattern | Complex rules |

### Real-World Examples

```kql
// SAP hierarchy filtering with has_prefix
GetSCIMIncidentV2
| where SAPPath has_prefix "CDOM > CDOM_Productivity"  // Any product under this CDOM
| summarize CaseCount = dcount(ServiceRequestNumber) by SAPPath

// File type filtering
GetSCIMIncidentV2
| where Attachment endswith ".log"  // Log files only
| summarize count()

// URL prefix matching
GetSCIMIncidentV2
| where CustomerEndpoint startswith "https://office365.com"
| summarize count()

// Exclude Microsoft internal paths
GetSCIMIncidentV2
| where !SAPPath startswith "CDOM > Internal"
| summarize CaseCount = dcount(ServiceRequestNumber) by Status
```

**Best Practice**: Use `has_prefix` for hierarchical data (SAP, CDOM), `startswith` for technical patterns (URLs, file extensions)

---
## 7. Join Types Overview

KQL supports 9 join types, each with different retention semantics:

| Join Type | Left Table | Right Table | Performance | Use Case |
|-----------|-----------|-----------|-------------|----------|
| **inner** | Matched | Matched | Fast | Correlation required |
| **leftouter** | All | Matched | Medium | Include unmatched left |
| **rightouter** | Matched | All | Medium | Include unmatched right |
| **fullouter** | All | All | Slowest | Full outer join |
| **leftanti** | Unmatched | None | Medium | Exclude joins |
| **leftsemi** | Matched | None | Fast | Filter by right table |
| **rightanti** | None | Unmatched | Medium | Exclude right unmatched |
| **rightsemi** | None | Matched | Fast | Filter by left on right |
| **cross** | All | All | Very slow | Cartesian product |

### Golden Rules
1. **Put the larger table on the LEFT**: KQL optimizes for left table size
2. **Filter BEFORE joining**: Reduce right table first
3. **Use semi/anti joins**: Faster than inner + distinct
4. **Avoid cross joins**: Only for <100 rows

### Join Syntax

```kql
LeftTable
| join kind=JOINTYPE (RightTable) on JoinKey
```

**Note**: Default join kind is `inner` if not specified

---
## 8. Inner & Outer Joins

### `inner` join - Only matched rows
- **Behavior**: Returns rows where both tables have matching keys
- **Performance**: Fast (optimized)
- **Result Shape**: Fewer rows than input

```kql
// Get support cases with matching ICM incidents (both exist)
GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| join kind=inner (
    cluster('icmcluster').database('IcmDataWarehouse').IncidentCustomFieldEntries
    | where FieldName == "LinkedSCIMId"
    | project IncidentId, LinkedSCIMId = Value
) on $left.ServiceRequestNumber == $right.LinkedSCIMId
| project ServiceRequestNumber, IncidentId, CreatedDateTime, Status
| take 100
```

### `leftouter` join - All left + matched right
- **Behavior**: Returns ALL rows from left table; adds right columns for matches (nulls otherwise)
- **Performance**: Medium (must preserve left rows)
- **Use Case**: Enrichment without data loss

```kql
// Get ALL support cases, with incident info if available
GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| join kind=leftouter (
    cluster('icmcluster').database('IcmDataWarehouse').Incidents
    | project IncidentId, IncidentSeverity = Severity, IncidentStatus = Status
) on $left.LinkedIncidentId == $right.IncidentId
| project ServiceRequestNumber, IncidentId, IncidentSeverity, Status
| summarize CaseCount = dcount(ServiceRequestNumber) by IncidentSeverity
```

**Key Difference**:
- `inner`: 95 cases + 87 incidents → 82 rows (only matched)
- `leftouter`: 95 cases + 87 incidents → 95 rows (all cases, some with null incident)

---
## 9. Anti & Semi Joins

### `leftanti` join - Left rows WITHOUT right matches
- **Behavior**: Returns left table rows that DON'T match right table
- **Performance**: Medium (exclusion filter)
- **Use Case**: Find orphaned/unlinked records

```kql
// Find support cases WITHOUT matching incidents
GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| join kind=leftanti (
    cluster('icmcluster').database('IcmDataWarehouse').Incidents
    | project LinkedSCIMId = CustomFields | where CustomFields has "ServiceRequestNumber"
    | distinct LinkedSCIMId
) on $left.ServiceRequestNumber == $right.LinkedSCIMId
| summarize UnopenedCases = dcount(ServiceRequestNumber) by Status
```

### `leftsemi` join - Left rows WITH right matches (fast)
- **Behavior**: Returns left table rows where right table HAS matching keys
- **Performance**: Fast (doesn't preserve right columns)
- **Use Case**: Filter left by right's existence

```kql
// Find cases that HAVE matching incidents (faster than inner)
GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| join kind=leftsemi (
    cluster('icmcluster').database('IcmDataWarehouse').Incidents
    | distinct LinkedSCIMId
) on $left.ServiceRequestNumber == $right.LinkedSCIMId
| summarize LinkedCaseCount = dcount(ServiceRequestNumber) by Status
```

### Performance Comparison

```
inner join:     ✓ Returns right columns | ✓ Fast | ✓ For correlations
leftsemi join:  ✗ No right columns | ✓ Faster | ✓ For existence checks
leftanti join:  ✗ No right columns | ✓ Medium | ✓ For exclusions
```

**Rule of Thumb**: If you don't need right table columns, use `leftsemi` (50% faster)

---
## 10. Join Performance & Best Practices

### Performance Optimization Checklist

1. **Filter Right Table First** (10x impact)
   ```kql
   // ✗ SLOW: 50M rows on right
   GetSCIMIncidentV2 | join (Incidents) on LinkedIncidentId
   
   // ✓ FAST: 5K rows on right
   GetSCIMIncidentV2
   | join (Incidents | where CreateDate > ago(30d)) on LinkedIncidentId
   ```

2. **Project Only Needed Columns** (5x impact)
   ```kql
   // ✗ SLOW: 200 columns from Incidents
   Incidents | join (GetSCIMIncidentV2) on ...
   
   // ✓ FAST: 5 columns from Incidents
   Incidents | project IncidentId, Status, Severity, Owner, Component
   | join (GetSCIMIncidentV2) on IncidentId
   ```

3. **Large Table on LEFT, Small on RIGHT**
   ```kql
   // ✓ FAST: 100K on left, 5K on right
   GetSCIMIncidentV2
   | join (Incidents | where Severity in (1,2)) on IncidentId
   
   // ✗ SLOW: 5K on left, 100K on right (reversed)
   (Incidents | where Severity in (1,2))
   | join (GetSCIMIncidentV2) on IncidentId
   ```

4. **Use Appropriate Join Type**
   ```kql
   // ✓ Fast: Only need existence check
   GetSCIMIncidentV2 | join kind=leftsemi (HighPriorityIncidents) on IncidentId
   
   // ✓ Fast: Only need exclusion
   GetSCIMIncidentV2 | join kind=leftanti (ClosedIncidents) on IncidentId
   
   // ⚠ Medium: Need enrichment
   GetSCIMIncidentV2 | join kind=leftouter (IncidentDetails) on IncidentId
   ```

### Multi-Table Join Pattern (FY25 Standard)

```kql
// Efficient multi-table enrichment: SCIM → ICM → Custom Fields
let CriticalIncidents = cluster('icmcluster').database('IcmDataWarehouse').Incidents
  | where CreateDate > ago(30d)
  | where Severity in (1, 2)  // Critical/High only
  | project IncidentId, IncidentSeverity = Severity, Status, OwningTeamId;

let CustomFields = cluster('icmcluster').database('IcmDataWarehouse').IncidentCustomFieldEntries
  | where FieldName in ("LinkedSCIMId", "CustomerImpact", "RootCause")
  | project IncidentId, FieldName, Value
  | evaluate pivot(FieldName, any(Value));

GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| project ServiceRequestNumber, DerivedProductName, AgentAlias, Severity
| join kind=leftouter CriticalIncidents on $left.ServiceRequestNumber == $right.LinkedSCIMId
| join kind=leftouter CustomFields on IncidentId
| summarize
    CaseCount = dcount(ServiceRequestNumber),
    LinkedIncidents = dcount(IncidentId),
    AvgSeverity = avg(toint(Severity))
    by DerivedProductName, AgentAlias
| sort by CaseCount desc
```

**Key Optimizations**:
1. Filter ICM tables by date/severity FIRST (reduces from 5M to 50K rows)
2. Project only needed columns upfront (200+ → 5 columns)
3. Use `let` to avoid re-filtering repeated subqueries
4. Apply `leftouter` for enrichment (preserve all SCIM cases)
5. Pivot custom fields once, not per query

---
## 11. Practical Real-World Examples

### Example 1: Week-over-Week Support Case Trends (Fiscal Calendar)

**Business Question**: Show support cases closed by week for last 8 weeks, normalized to fiscal calendar

**Fiscal Calendar**: FY starts July 1; Week 1 = Jul 1-7, Week 2 = Jul 8-14, etc.

```kql
let FY_START = datetime(2024-07-01);
let FY_END = datetime(2025-06-30);

GetSCIMIncidentV2
| where ClosedDateTime between (FY_START .. FY_END)
| where DaysToClose >= 0  // Valid closures only
| extend
    DayOfWeek = dayofweek(ClosedDateTime),
    FiscalWeek = tolong((datetime_diff('day', ClosedDateTime, FY_START)) / 7) + 1
| summarize
    CasesClosed = dcount(ServiceRequestNumber),
    AvgDaysToClose = avg(DaysToClose),
    AvgSeverity = avg(toint(Severity))
    by FiscalWeek
| sort by FiscalWeek desc
```

**Output**: Fiscal weeks 1-52 with closure metrics for trend analysis

---

### Example 2: Engineer Performance Report (Manager Rollup)

**Business Question**: For each manager, show team-level case metrics (closure rate, SLA compliance, severity distribution)

```kql
GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| where DaysToClose >= 0  // Completed cases
| extend
    SLAMet = iff(DaysToClose <= 3, 1, 0),  // SLA is 3 days
    SeverityRank = case(
        Severity in ("1", "A"), 1,  // Critical
        Severity in ("2", "B"), 2,  // High
        Severity in ("3", "C"), 3,  // Medium
        4)  // Low
| summarize
    TotalCases = dcount(ServiceRequestNumber),
    SLACompliance = round(100.0 * sum(SLAMet) / dcount(ServiceRequestNumber), 2),
    AvgDaysToClose = round(avg(DaysToClose), 1),
    P95DaysToClose = percentile(DaysToClose, 95),
    CriticalCases = sumif(SeverityRank, SeverityRank == 1)
    by ManagerEmail, AgentAlias
| project
    Manager = ManagerEmail,
    Engineer = AgentAlias,
    Cases = TotalCases,
    SLA = SLACompliance,
    AvgDays = AvgDaysToClose,
    Critical = CriticalCases
| sort by SLA asc  // Show lowest performers first
```

**Output**: Manager → Engineer → performance metrics

---

### Example 3: Multi-Cluster Incident Enrichment (Graph Semantics)

**Business Question**: Find support cases with matching ICM incidents, enriched with root cause and customer impact

```kql
let IcmCritical = cluster('icmcluster').database('IcmDataWarehouse').Incidents
  | where CreateDate > ago(7d)
  | where Severity in (1, 2)  // Critical/High
  | where Status != "RESOLVED"
  | project IncidentId, IcmSeverity = Severity, Status, OwningTeamName;

let IncidentLinks = cluster('icmcluster').database('IcmDataWarehouse').IncidentCustomFieldEntries
  | where FieldName == "LinkedSCIMServiceRequestNumber"
  | project IncidentId, LinkedSCIMId = Value;

GetSCIMIncidentV2
| where CreatedDateTime > ago(7d)
| where Severity in ("1", "2")  // Match ICM severity
| project ServiceRequestNumber, DerivedProductName, Title, CreatedDateTime, Status
| join kind=inner IncidentLinks on $left.ServiceRequestNumber == $right.LinkedSCIMId
| join kind=leftouter IcmCritical on IncidentId
| summarize
    CaseCount = dcount(ServiceRequestNumber),
    LinkedIncidents = dcount(IncidentId),
    Teams = dcount(OwningTeamName)
    by DerivedProductName, Status
| sort by LinkedIncidents desc
```

**Key Pattern**: `let` bindings for filtered subsets → progressive joins → aggregation by business dimensions

---

### Example 4: Last-Known State via Materialized Views (Performance)

**Problem**: Querying current incident state repeatedly requires expensive aggregations

**Solution**: Pre-compute latest state snapshot; update materialized view hourly

```kql
// Define materialized view (admin command)
.create materialized-view with (folder='Incidents', docstring='Current incident state snapshot') IncidentState_MV on table Incidents
{
    Incidents
    | summarize
        arg_max(LastModifiedDate, *)
        by IncidentId
}

// Create convenience function
.create function with (folder='Incidents') GetCurrentIncidents(severity: int) {
    IncidentState_MV
    | where Severity == severity
    | where Status != "RESOLVED"
}

// Use: Fast query on pre-computed snapshot
GetCurrentIncidents(1)  // All current critical incidents
| summarize count() by OwningTeamName
```

**Performance Gain**: 100-1000x faster (pre-computed vs. on-demand aggregation)

---

### Example 5: Natural Language to KQL Translation

**User Request**: "Show me high-severity cases from last 2 weeks, grouped by product, sorted by count"

**Translation Process**:

1. **Identify Time Scope**: "last 2 weeks" → `CreatedDateTime > ago(14d)`
2. **Extract Entity Filters**: "high-severity" → `Severity in ("1", "A", "2", "B")`
3. **Decompose Predicates**: Single condition (high severity) → simple `where` clause
4. **Specify Aggregations**: "grouped by product" → `by DerivedProductName` + `dcount(ServiceRequestNumber)`
5. **Apply Refinements**: "sorted by count" → `sort by CaseCount desc`

**Final Query**:

```kql
GetSCIMIncidentV2
| where CreatedDateTime > ago(14d)
| where Severity in ("1", "A", "2", "B")
| summarize CaseCount = dcount(ServiceRequestNumber) by DerivedProductName
| sort by CaseCount desc
```

---

### Example 6: Complex Multi-Criteria Query with Pivoting

**User Request**: "For each product, show how many cases are open, in-progress, and resolved by severity level"

```kql
GetSCIMIncidentV2
| where CreatedDateTime > ago(30d)
| summarize CaseCount = dcount(ServiceRequestNumber) by DerivedProductName, Status, Severity
| evaluate pivot(Status, sum(CaseCount))  // Convert Status → columns
| project
    Product = DerivedProductName,
    Severity,
    Open = ['Open'],
    InProgress = ['In Progress'],
    Resolved = ['Resolved']
```

**Output**: Matrix view of products × severity with status distribution

---
## Quick Reference: Common KQL Patterns

### DateTime Functions
```kql
ago(7d)                           // 7 days ago from now
datetime(2024-12-15)              // Specific datetime
datetime_add('day', -30, now())   // 30 days back
dayofweek(timestamp)              // Day of week (0=Sunday)
bin(timestamp, 1d)                // Round down to day
startofday(timestamp)             // Start of day (00:00:00)
endofday(timestamp)               // End of day (23:59:59)
```

### Aggregation Functions
```kql
count()                          // Row count
dcount(column)                   // Distinct count
sum(numeric_column)              // Sum values
avg(numeric_column)              // Average
percentile(column, percentile)   // Percentile (0-100)
arg_max(timestamp, *)            // Row with max timestamp
max(column)                      // Maximum value
min(column)                      // Minimum value
```

### Type Conversion
```kql
tostring(value)        // Convert to string
toint(value)           // Convert to int
toreal(value)          // Convert to double
todatetime(value)      // Convert to datetime
tolower(string)        // Lowercase
toupper(string)        // Uppercase
```

### String Functions
```kql
extract(pattern, group, string)      // Extract regex group
replace_string(haystack, needle, replacement)  // Replace substring
substring(string, start, length)     // Extract substring
strlen(string)                       // String length
split(string, delimiter)             // Split into array
strcat(str1, str2, ...)              // Concatenate strings
```

---
## Summary: When to Use Each Operator

| Question | Operator | Example |
|----------|----------|----------|
| Does field contain word? | `has` | `Title has "timeout"` |
| Does field contain substring? | `contains` | `Title contains "sql"` |
| Match exact status? | `==` | `Status == "Active"` |
| Does field start with? | `startswith` or `has_prefix` | `Path has_prefix "CDOM > Prod"` |
| Need complex pattern? | `matches regex` | `Code matches regex "ERR-\d{4}"` |
| Is value in list? | `in()` | `Status in ("Active", "Open")` |
| Find matching rows in both? | `join kind=inner` | Multi-table correlation |
| Keep unmatched left rows? | `join kind=leftouter` | Enrichment with nulls |
| Find unmatched left rows? | `join kind=leftanti` | Exclusion filter |
| Fast existence check? | `join kind=leftsemi` | Filter by presence |

---

## Performance Hierarchy (Fastest to Slowest)

1. **Equality** (`==`, `!=`) - Indexed, <10ms
2. **Word match** (`has`, `has_prefix`) - Indexed, 10-100ms
3. **Prefix match** (`startswith`, `endswith`) - Indexed, 10-100ms
4. **Semi/Anti joins** - Medium, 100-500ms
5. **Substring** (`contains`) - Not indexed, 100-1000ms
6. **Case-insensitive** (`=~`) - Not indexed, 100-1000ms
7. **Inner/Outer joins** - Medium, 100-1000ms
8. **Regex** (`matches regex`) - Very slow, 500-5000ms
9. **Full outer join** - Slowest, 1000ms+
10. **Cross join** - Dangerous, potential cartesian explosion

**Rule**: Always filter before expensive operations (joins, regex, contains)